<a href="https://colab.research.google.com/github/dantae74/web-crawling/blob/main/6_%EB%8F%99%EC%A0%81_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91(%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%89%B4%EC%8A%A4_%EA%B8%B0%EC%82%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 동적 데이터 수집
- 네이버 기사 여러 페이지 수집하기

---

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

## 0. 라이브러리 import

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd

## 1. 네이버 뉴스 검색 결과로 바로 이동하기

- www.naver.com 으로 이동하여 검색하는 방법도 있지만
- 바로 이동할 수 있는 방법이 있다면 시간과 자원을 절약할 수 있음

In [ ]:
# 페이지 접근
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=KT WIZ') # 크롤링 예제 사이트

## 2. 페이지 탐색 및 크롤링 준비

In [ ]:
# 뉴스 기사가 모아놓은 요소 찾기
articles_box = driver.find_element(By.CSS_SELECTOR,'#main_pack > section > div > div.group_news > ul')
# 뉴스 기사 리스트 찾기
article_list = articles_box.find_elements(By.CSS_SELECTOR,'li.bx')
print(len(article_list))
#sp_nws1, #sp_nws1 > div.news_wrap.api_ani_send

In [ ]:
for article in article_list:
    # 뉴스 제목 가져오기
    title = article.find_element(By.CSS_SELECTOR,'a.news_tit').text
    print("뉴스 제목 : " + title)

    # 뉴스 설명 가져오기
    dsc = article.find_element(By.CSS_SELECTOR, "div.dsc_wrap > a").text
    print("뉴스 내용 : " + dsc)

    # 신문사 가져오기
    press = article.find_element(By.CSS_SELECTOR, "a.press").text
    print(press)


In [ ]:
# 페이지 번호 요소 찾기
page_wrap = driver.find_element(By.CSS_SELECTOR, 'div.sc_page')

# 찾은 요소에서 html 코드 가져오기
page_wrap_html = page_wrap.get_attribute('innerHTML')
page_soup = BeautifulSoup(page_wrap_html, 'html.parser')
print(page_soup.prettify())

- disabled 속성으로 이전페이지가 있는지, 이후페이지가 있는지 알 수 있다.
- pressed 속성으로 현재 페이지가 몇인지도 알 수 있다.

In [ ]:
# 현재 페이지 찾기
# aria-pressed="true" 인 요소 찾기
now_page = page_wrap.find_element(By.CSS_SELECTOR, 'a[aria-pressed="true"]')
print(now_page.text)

# 다음 페이지 이동 불가능 여부 가져오기
next_page = page_wrap.find_element(By.CSS_SELECTOR, 'a.btn_next')
aria_disabled = next_page.get_attribute('aria-disabled')
print(aria_disabled)

- 현재 페이지의 뉴스 제목, 내용, 신문사를 출력하는 함수 작성

In [ ]:
def get_news_info(driver):
    result = []
    # 뉴스 기사들을 모아 놓은 요소가 로딩될때 까지 대기
    articles_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main_pack > section > div > div.group_news > ul')))
    # 뉴스 기사 리스트 찾기
    article_list = articles_box.find_elements(By.CSS_SELECTOR,'li.bx')
    for article in article_list:
        news_info = {}
        # 뉴스 제목 가져오기
        title = article.find_element(By.CSS_SELECTOR,'a.news_tit').text
        news_info['제목'] = title
        # 뉴스 내용 가져오기
        dsc = article.find_element(By.CSS_SELECTOR, "div.dsc_wrap > a").text
        news_info['내용'] = dsc
        # 신문사 가져오기
        press = article.find_element(By.CSS_SELECTOR, "a.press").text
        news_info['언론사'] = press

        result.append(news_info)
    return result


- 여러 페이지 크롤링을 위해 페이지 이동할때마다 현재 페이지 정보와, 다음 페이지 이동 가능 여부를 판단하는 함수 작성

In [ ]:
def get_page_info(driver):
    result = {}
    # 페이지 이동마다 사용하기 때문에 Wait 사용
    page_wrap = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR , 'div.sc_page')))
    # 현재 페이지 찾기
    now_page = page_wrap.find_element(By.CSS_SELECTOR, 'a[aria-pressed="true"]').text
    result['현재 페이지'] = now_page
    # 다음 페이지 이동 불가능 여부 가져오기
    next_page = page_wrap.find_element(By.CSS_SELECTOR, 'a.btn_next')
    result['다음 페이지'] = next_page
    aria_disabled = next_page.get_attribute('aria-disabled')
    result['다음 페이지 이동 불가'] = aria_disabled


    return result

## 3. 크롤링

In [ ]:
# 드라이버 로드
# driver = webdriver.Chrome()
driver = webdriver.Chrome(options=options)

# 페이지 접근
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=KT WIZ') # 크롤링 예제 사이트

df = []

# 최대 10페이지까지 크롤링
for i in range(0,10):
    news_info = get_news_info(driver)
    page_info = get_page_info(driver)
    print(page_info['현재 페이지'])
    print(news_info)
    for news in news_info:
        df.append(news)
    # 다음 페이지로 이동
    if page_info['다음 페이지 이동 불가'] == 'true':
        break
    else:
        page_info['다음 페이지'].click()

driver.quit()

In [ ]:
pd.DataFrame(df).head(30)